# Aspect Extraction with attention neural network

## ABSA 관련된 코드를 아직 작성 중에 있습니다. 따라서 제출 코드는 사용할 라이브러리와 전처리 및 토크나이징 과정을 담은 코드를 우선적으로 제출하겠습니다.

In [1]:

# https://towardsdatascience.com/case-study-2-an-unsupervised-neural-attention-model-for-aspect-extraction-1c2c97b1380a
# https://github.com/Ashcom-git/case-study-2/blob/main/case_study_2.ipynb

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from tqdm import tqdm
import nltk
import numpy as np
from gensim.models import Word2Vec
import tensorflow as tf
from sklearn.cluster import KMeans
from sklearn.decomposition import LatentDirichletAllocation
from tensorflow.keras.layers import Layer, Embedding, Input, Dense, Activation
from tensorflow.keras.initializers import Constant
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import Model,regularizers, initializers, constraints
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.data import Dataset
import tensorflow.keras.backend as K
from tensorflow import lite
import pickle
import sys

## Preprocessing

In [7]:
stop = stopwords.words('english')
lemma = WordNetLemmatizer()


#text_token 을 만들고 -> lemmatizing 하고 -> 다시 문장으로 바꾸고

def preprocess(text):

  text_token = CountVectorizer().build_tokenizer()(text.lower())
  text_token = [lemma.lemmatize(i) for i in text_token if i not in stop]
  if len(text_token)>0:
    return ' '.join(text_token)+'\n'
  return '\n'  

def complete_preprocess(list_):

  processed_lst = []
  for i in tqdm(list_, position=0):
    pre = preprocess(i)
    processed_lst.append(pre) 
 
  return processed_lst

### training word2vec

In [10]:
def split_list(list_):
  
  split_ = []
  for i in list_:
    split_.append(i.split())
  return split_   

sp_review = split_list(pre_laptop_review)
# sp_train = split_list(pre_laptop_train)
# sp_test = split_list(pre_laptop_test)

## tokenizing / padding

In [14]:
#num, unk 토큰 추가

def read_dataset( phase, vocab):

    num_hit, unk_hit, total = 0, 0, 0
    data_lst = []

    for line in phase:
        if len(line) > 0:
          indices = []
          for word in line:
              if word.replace('.','',1).isdigit():
                  indices.append(vocab['<num>'])
                  num_hit += 1
              elif word in vocab:
                  indices.append(vocab[word])
              else:
                  indices.append(vocab['<unk>'])
                  unk_hit += 1
              total += 1
          data_lst.append(indices)

    if len(phase)>30000:
      set_ = 'train'
    else:
      set_ = 'test'  
    print('{} set - <num> hit rate: {}%, <unk> hit rate: {}%'.format(set_,round(100*(num_hit/total),2), round(100*(unk_hit / total),2)))
    return data_lst

In [15]:
#단어 사전 만들기
total_words, unique_words = 0, 0
word_freqs = {} #key = 단어, value = 빈도수
for line in sp_review:
    for w in line:
        if w in word_freqs.keys():
            word_freqs[w] += 1
        else:
            word_freqs[w] = 1
            unique_words += 1
        total_words += 1
        
print('Total words: {}, Unique words: {}'.format(total_words,unique_words))

sorted_word_freqs = sorted(word_freqs.items(), key=lambda x : x[1], reverse = True)
vocab_dict = {'<pad>':0,'<unk>':1,'<num>':2}
# sorted_word_freqs
idx_words = {j[0]:i+3 for i,j in enumerate(sorted_word_freqs)}
vocab_dict.update(idx_words) #단어사전 완성

review_sequence = read_dataset(sp_review, vocab_dict)
# train_sequence = read_dataset(sp_train, vocab_dict)
# test_sequence = read_dataset(sp_test, vocab_dict) #정수 인코딩 및 토큰 부여 완성


review_max = max(map(lambda x:len(x), review_sequence))
# review_max = max(map(lambda x:len(x), train_sequence))
# train_max = max(map(lambda x:len(x), train_sequence))
# test_max = max(map(lambda x:len(x), test_sequence))

review_sequence = pad_sequences(review_sequence, maxlen = review_max)
# train_sequence = pad_sequences(train_sequence, maxlen=max(train_max,test_max))
# test_sequence = pad_sequences(test_sequence, maxlen=max(train_max,test_max)) #단어 패딩 완료(pre:0)


Total words: 1191382, Unique words: 23152
train set - <num> hit rate: 1.26%, <unk> hit rate: 0.0%
